In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [6]:
# convert csv to dataframe
df_cleaned=pd.read_csv(r"E:\E-commerce_project\dataset ecom\combined_data.csv")
df_cleaned

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,1.0,credit_card,1.0,18.12,NaN,NaN,NaN,NaN,NaN,NaN
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,3.0,voucher,1.0,2.00,NaN,NaN,NaN,NaN,NaN,NaN
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,2.0,voucher,1.0,18.59,NaN,NaN,NaN,NaN,NaN,NaN
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,1.0,boleto,1.0,141.46,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,1.0,credit_card,3.0,179.12,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118441,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,da62f9e57a76d978d02ab5362c509660,11722,...,1.0,credit_card,3.0,195.00,NaN,NaN,NaN,NaN,NaN,NaN
118442,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,737520a9aad80b3fbbdad19b66b37b30,45920,...,1.0,credit_card,5.0,271.01,NaN,NaN,NaN,NaN,NaN,NaN
118443,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,...,1.0,credit_card,4.0,441.16,NaN,NaN,NaN,NaN,NaN,NaN
118444,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,5097a5312c8b157bb7be58ae360ef43c,28685,...,1.0,credit_card,4.0,441.16,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Drop useless columns
df_model = df_cleaned.drop(columns=[
    'order_id', 'customer_id', 'product_id', 'seller_id', 
    'review_id', 'review_creation_date', 'product_category_name_english'
])

In [8]:
# Function to calculate delay in days
def calculate_delay(row):
    """Calculate the delay between estimated and actual delivery date in days"""
    if pd.isnull(row['order_delivered_customer_date']):
        return np.nan
    
    estimated_date = pd.to_datetime(row['order_estimated_delivery_date'])
    delivered_date = pd.to_datetime(row['order_delivered_customer_date'])
    
    # Negative values mean early delivery, positive values mean delay
    delay = (delivered_date - estimated_date).days
    
    return delay

# Data Preparation
print("Preparing data for model training...")

Preparing data for model training...


In [9]:
# Convert datetime columns
datetime_columns = [
    'order_purchase_timestamp', 
    'order_approved_at', 
    'order_delivered_carrier_date', 
    'order_delivered_customer_date', 
    'order_estimated_delivery_date',
    'shipping_limit_date',
    'review_creation_date',
    'review_answer_timestamp'
]

for col in datetime_columns:
    if col in df_cleaned.columns:
        df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')

# Calculate delivery delay for each order
df_cleaned['delivery_delay'] = df_cleaned.apply(calculate_delay, axis=1)

# Filter for delivered orders only to have valid target values
delivered_orders = df_cleaned[df_cleaned['order_status'] == 'delivered'].copy()

# Calculate time intervals
delivered_orders['approval_time'] = (delivered_orders['order_approved_at'] - 
                                    delivered_orders['order_purchase_timestamp']).dt.total_seconds() / 3600  # in hours
delivered_orders['carrier_time'] = (delivered_orders['order_delivered_carrier_date'] - 
                                   delivered_orders['order_approved_at']).dt.total_seconds() / 3600  # in hours
delivered_orders['estimated_wait_time'] = (delivered_orders['order_estimated_delivery_date'] - 
                                          delivered_orders['order_purchase_timestamp']).dt.total_seconds() / (3600 * 24)  # in days

# Create distance features
delivered_orders['same_state'] = (delivered_orders['seller_state'] == delivered_orders['customer_state']).astype(int)
delivered_orders['same_city'] = (delivered_orders['seller_city'] == delivered_orders['customer_city']).astype(int)

# Calculate product volume
delivered_orders['product_volume_cm3'] = (delivered_orders['product_length_cm'] * 
                                         delivered_orders['product_height_cm'] * 
                                         delivered_orders['product_width_cm'])

In [10]:
# Handle missing values in product features
for col in ['product_weight_g', 'product_volume_cm3']:
    delivered_orders[col].fillna(delivered_orders[col].median(), inplace=True)

# Extract month, day of week, and hour from purchase timestamp for seasonality
delivered_orders['purchase_month'] = delivered_orders['order_purchase_timestamp'].dt.month
delivered_orders['purchase_day_of_week'] = delivered_orders['order_purchase_timestamp'].dt.dayofweek
delivered_orders['purchase_hour'] = delivered_orders['order_purchase_timestamp'].dt.hour

# Create payment features
delivered_orders['total_payment'] = delivered_orders['payment_value']
delivered_orders['payment_installments'] = delivered_orders['payment_installments'].fillna(1)

# Group data by order_id to handle multiple items per order
# First, select features that need to be aggregated
item_features = [
    'order_item_id', 'price', 'freight_value', 
    'product_weight_g', 'product_volume_cm3', 
    'product_photos_qty', 'payment_value'
]

# Aggregate data at order level
order_agg = delivered_orders.groupby('order_id').agg({
    'order_item_id': 'count',                # Number of items in the order
    'price': 'sum',                          # Total price of the order
    'freight_value': 'sum',                  # Total freight value
    'product_weight_g': 'sum',               # Total weight
    'product_volume_cm3': 'sum',             # Total volume
    'product_photos_qty': 'mean',            # Average photos per product
    'payment_installments': 'max',           # Max installments
    'payment_value': 'sum',                  # Total payment value
    'review_score': 'first'                  # Review score (assuming one per order)
}).reset_index()
# Get features that should be unique per order
order_features = delivered_orders.drop_duplicates(subset='order_id')

# Select columns that should be unique per order
unique_columns = [
    'order_id', 'customer_id', 'order_status', 
    'order_purchase_timestamp', 'order_approved_at', 
    'order_delivered_carrier_date', 'order_delivered_customer_date', 
    'order_estimated_delivery_date', 'customer_zip_code_prefix', 
    'customer_city', 'customer_state', 'seller_id', 
    'seller_zip_code_prefix', 'seller_city', 'seller_state',
    'same_state', 'same_city', 'purchase_month', 
    'purchase_day_of_week', 'purchase_hour',
    'approval_time', 'carrier_time', 'estimated_wait_time', 
    'delivery_delay', 'payment_type'
]

# Select only columns that exist in order_features
existing_columns = [col for col in unique_columns if col in order_features.columns]
order_features = order_features[existing_columns]

# Merge aggregated data with order features
final_orders = order_features.merge(order_agg, on='order_id', how='left')

In [11]:
# Select features for training
feature_columns = [
    'purchase_month', 'purchase_day_of_week', 'purchase_hour',
    'approval_time', 'carrier_time', 'estimated_wait_time',
    'same_state', 'same_city', 'order_item_id',  # order_item_id represents item count after aggregation
    'price', 'freight_value', 'product_weight_g', 'product_volume_cm3',
    'payment_installments', 'payment_value'
]

# Make sure all feature columns exist in the dataset
feature_columns = [col for col in feature_columns if col in final_orders.columns]

# Prepare features and target
X = final_orders[feature_columns].copy()
y = final_orders['delivery_delay'].copy()

# Handle missing values
for col in X.columns:
    if X[col].isnull().sum() > 0:
        if X[col].dtype in ['int64', 'float64']:
            X[col].fillna(X[col].median(), inplace=True)
        else:
            X[col].fillna(X[col].mode()[0], inplace=True)

# Add payment_type as a feature if it exists
if 'payment_type' in final_orders.columns:
    payment_dummies = pd.get_dummies(final_orders['payment_type'], prefix='payment')
    X = pd.concat([X, payment_dummies], axis=1)

In [12]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Training set shape: (77182, 19)
Testing set shape: (19296, 19)


In [13]:
# Define categorical and numerical features
categorical_features = ['purchase_month', 'purchase_day_of_week', 'purchase_hour', 'same_state', 'same_city']
categorical_features = [col for col in categorical_features if col in X.columns]

numerical_features = [col for col in X.columns if col not in categorical_features 
                      and not col.startswith('payment_')]  # Exclude dummy variables

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'  # This will pass through the payment dummy variables
)

In [14]:
# Train and evaluate multiple models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Create pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Train model
    pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_pred = pipeline.predict(X_test)
    
    # Evaluate
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'pipeline': pipeline
    }
    
    print(f"  MAE: {mae:.2f} days")
    print(f"  RMSE: {rmse:.2f} days")
    print(f"  R²: {r2:.4f}")


Training Linear Regression...
  MAE: 4.61 days
  RMSE: 7.66 days
  R²: 0.4296

Training Random Forest...
  MAE: 4.54 days
  RMSE: 7.68 days
  R²: 0.4269

Training Gradient Boosting...
  MAE: 4.47 days
  RMSE: 7.59 days
  R²: 0.4391


In [15]:
# Find the best model
best_model = min(results.items(), key=lambda x: x[1]['MAE'])
print(f"\nBest model: {best_model[0]} with MAE: {best_model[1]['MAE']:.2f} days")


Best model: Gradient Boosting with MAE: 4.47 days


In [16]:
# Hyperparameter tuning for the best model
print("\nPerforming hyperparameter tuning for Gradient Boosting...")
param_grid = {
    'model__n_estimators': [100, 200],
    'model__learning_rate': [0.05, 0.1],
    'model__max_depth': [3, 5]
}


Performing hyperparameter tuning for Gradient Boosting...


In [17]:
# Create pipeline with Gradient Boosting
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(random_state=42))
])

# Grid search
grid_search = GridSearchCV(
    gb_pipeline,
    param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")

Best parameters: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 200}


In [18]:
# Evaluate the tuned model
best_gb = grid_search.best_estimator_
y_pred_tuned = best_gb.predict(X_test)

mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
r2_tuned = r2_score(y_test, y_pred_tuned)

print(f"Tuned model performance:")
print(f"  MAE: {mae_tuned:.2f} days")
print(f"  RMSE: {rmse_tuned:.2f} days")
print(f"  R²: {r2_tuned:.4f}")

Tuned model performance:
  MAE: 4.34 days
  RMSE: 7.47 days
  R²: 0.4576


In [19]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize model
xgb_model = XGBRegressor(random_state=42)

# Fit
xgb_model.fit(X_train, y_train)

# Predict
y_pred = xgb_model.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

MAE: 4.44
RMSE: 7.56
R²: 0.4439


In [20]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.3],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=50,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [21]:
print("Best Hyperparameters:\n", random_search.best_params_)

Best Hyperparameters:
 {'subsample': 0.8, 'reg_lambda': 1.5, 'reg_alpha': 0.1, 'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.6}


In [22]:
print("Best CV R² Score: {:.4f}".format(random_search.best_score_))

Best CV R² Score: 0.4478


In [23]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

MAE: 4.34
RMSE: 7.49
R²: 0.4536


In [25]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 👇 Insert your tuned Gradient Boosting params here
gbr_best = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    random_state=42
)

# 👇 Use the best XGBoost params from random_search
xgb_best = XGBRegressor(**random_search.best_params_)

# Base models
base_models = [
    ('xgb', xgb_best),
    ('gbr', gbr_best)
]

# Meta model
meta_model = LinearRegression()

# Stacking model
stacked_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

# Fit model
stacked_model.fit(X_train, y_train)

# Predict
y_pred_stack = stacked_model.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred_stack)
rmse = mean_squared_error(y_test, y_pred_stack, squared=False)
r2 = r2_score(y_test, y_pred_stack)

print(f"Stacked MAE: {mae:.2f}")
print(f"Stacked RMSE: {rmse:.2f}")
print(f"Stacked R²: {r2:.4f}")

Stacked MAE: 4.33
Stacked RMSE: 7.44
Stacked R²: 0.4609


In [30]:
import joblib
joblib.dump(stacked_model, 'final_stacked_model.pkl')

['final_stacked_model.pkl']

In [ ]:
# Plot actual vs predicted delays
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_tuned, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Delay (days)')
plt.ylabel('Predicted Delay (days)')
plt.title('Actual vs Predicted Delivery Delays')
plt.tight_layout()
plt.show()

# Plot error distribution
plt.figure(figsize=(10, 6))
errors = y_test - y_pred_tuned
plt.hist(errors, bins=30)
plt.xlabel('Prediction Error (days)')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.axvline(x=0, color='r', linestyle='--')
plt.tight_layout()
plt.show()

# Save the model
import joblib
joblib.dump(best_gb, 'olist_delivery_delay_model.pkl')

print("\nModel saved as 'olist_delivery_delay_model.pkl'")
print("You can load it using: model = joblib.load('olist_delivery_delay_model.pkl')")

# Function to predict delivery delay for new orders
def predict_delivery_delay(new_data):
    """
    Predict delivery delay for new orders
    
    Parameters:
    new_data (DataFrame): DataFrame with the same features used for training
    
    Returns:
    array: Predicted delays in days
    """
    # Make sure new_data has all required columns
    required_cols = set(feature_columns)
    if not required_cols.issubset(set(new_data.columns)):
        missing = list(required_cols - set(new_data.columns))
        print(f"Missing columns: {missing}")
        raise ValueError("New data must contain all feature columns used for training")
    
    # If payment_type was used in training, ensure it's properly encoded
    if 'payment_type' in final_orders.columns:
        payment_types = final_orders['payment_type'].unique()
        for payment in payment_types:
            col_name = f'payment_{payment}'
            if col_name not in new_data.columns:
                new_data[col_name] = 0
            
    return best_gb.predict(new_data[X.columns])

print("\nModel training complete! Use the predict_delivery_delay() function to make predictions on new orders.")